# Generalizing a Taylor Recurrence

In [28]:
from sumpy.recurrence import _make_sympy_vec, get_processed_and_shifted_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_recurrence, recurrence_from_pde, shift_recurrence, get_shifted_recurrence_exp_from_pde

import sympy as sp
from sympy import hankel1

import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import cm, ticker

In [29]:
var = _make_sympy_vec("x", 2)
s = sp.Function("s")
n = sp.symbols("n")

In [30]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)

w = make_identity_diff_op(2)
helmholtz2d = laplacian(w) + w

In [31]:
def compute_derivatives(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
    derivs = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                        for i in range(p)]
    return derivs

In [32]:
def compute_derivatives_h2d(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    abs_dist = sp.sqrt((var[0]-var_t[0])**2 +
                        (var[1]-var_t[1])**2)
    g_x_y = (1j/4) * hankel1(0, k * abs_dist)
    derivs_helmholtz = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                                                for i in range(p)]
    return derivs_helmholtz

In [33]:
recur, order = get_shifted_recurrence_exp_from_pde(laplace2d)
order

4

In [34]:
#Sanity check that recurrence is correct
derivs_lap = compute_derivatives(5)
exp = recur.subs(n, 4)
exp.subs(s(4), derivs_lap[4]).subs(s(3), derivs_lap[3]).subs(s(2), derivs_lap[2]).subs(s(1), derivs_lap[1]).subs(var[0],np.random.rand()).subs(var[1],np.random.rand())

1.55431223447522e-15

## Step 2: We need to arrange the terms in the recurrence as a polynomial in $x_0$, $s(n)$
$$
table[i, j]
$$
Where $i = 0$ represents the coefficient attached to $s(n)$ and $i = 1$ represents $s(n-1)$, etc. and the $j$ is for the polynomial in $x_0$.

In [59]:
recur
poly_in_s_n = sp.poly(recur, [s(n-i) for i in range(order)])
poly_in_s_n

Poly(((-1)**n*x0**3 + (-1)**n*x0*x1**2)*(s(n)) + (-3*(-1)**n*n*x0**2 - (-1)**n*n*x1**2 + 5*(-1)**n*x0**2 + 3*(-1)**n*x1**2)*(s(n - 1)) + (3*(-1)**n*n**2*x0 - 13*(-1)**n*n*x0 + 14*(-1)**n*x0)*(s(n - 2)) + (-(-1)**n*n**3 + 8*(-1)**n*n**2 - 21*(-1)**n*n + 18*(-1)**n)*(s(n - 3)), s(n), s(n - 1), s(n - 2), s(n - 3), domain='ZZ[n,x0,(-1)**n,x1]')

In [ ]:
coeff_s_n = [poly_in_s_n.coeff_monomial(poly_in_s_n.gens[i]) for i in range(order)]

table = []
for i in range(len(coeff_s_n)):
    table.append(sp.poly(coeff_s_n[i], var[0]).all_coeffs()[::-1])

In [58]:
table

[[0, (-1)**n*x1**2, 0, (-1)**n],
 [-(-1)**n*n*x1**2 + 3*(-1)**n*x1**2, 0, -3*(-1)**n*n + 5*(-1)**n],
 [0, 3*(-1)**n*n**2 - 13*(-1)**n*n + 14*(-1)**n],
 [-(-1)**n*n**3 + 8*(-1)**n*n**2 - 21*(-1)**n*n + 18*(-1)**n]]